In [194]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [195]:
#import libraries
import cupy, cudf
from cuml.preprocessing import LabelEncoder
from cuml.model_selection import train_test_split
from cuml import LinearRegression
import cuml 

In [196]:
#importing datatsets
train = cudf.read_csv("/kaggle/input/ammi-bootcamp-kaggle-competition/train.csv")

In [197]:
#dropping na values 
train.drop(['description', 'title', 'id', ], axis=1, inplace=True)
train.dropna(axis=0, inplace=True)

In [198]:
#splitting into x and y 
x = train.drop(['price'], axis=1)
y = train['price']

In [199]:
#encoding
encoder = LabelEncoder()
labels=['country','designation','province','region_1','variety','winery','region_2','taster_name','taster_twitter_handle']
for a in labels:
    x[a] = encoder.fit_transform(x[a])

In [200]:
#standardizing
max_value = x.max()
for a in range(len(labels)+1):
    x.iloc[:,a] = x.iloc[:,a]/max_value[a]
x = x.fillna(0)

In [201]:
#splitting into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.1)

In [202]:
#model training and prediction
model=LinearRegression(algorithm="svd-jacobi")
model.fit(x_train,y_train)
pred = model.predict(x_test)

In [203]:
#metrics
print(cuml.metrics.regression.mean_squared_error(y_test,pred))
print(cuml.metrics.regression.mean_absolute_error(y_test,pred))
print(cuml.metrics.regression.r2_score(y_test,pred))